# Model Training: SARIMA (Global + Per-Product)

Ce notebook implémente deux approches SARIMA :
1. **Globale** : Modèle unique sur les ventes agrégées.
2. **Par Produit** : Modèle individuel pour chaque produit.

Données utilisées : `Data/train_sarimax.csv` et `Data/test_sarimax.csv`.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import os
import warnings

warnings.filterwarnings('ignore')

os.makedirs('trained_models', exist_ok=True)
print("✅ Bibliothèques importées.")

/home/raph/.local/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


✅ Bibliothèques importées.


## Chargement des Données

In [2]:
train_path = 'Data/train_sarimax.csv'
test_path = 'Data/test_sarimax.csv'

if not os.path.exists(train_path) or not os.path.exists(test_path):
    raise FileNotFoundError("Fichiers Data/ introuvables.")

train_df = pd.read_csv(train_path, parse_dates=['Date'])
test_df = pd.read_csv(test_path, parse_dates=['Date'])

print(f"📦 Train cols: {list(train_df.columns)}")
print(f"📦 Train shape: {train_df.shape}")

📦 Train cols: ['Product ID', 'Date', 'Units Sold', 'Price', 'Discount', 'Holiday/Promotion', 'Competitor Pricing']
📦 Train shape: (1680, 7)


## 1. Modèle Global
Nous agrégeons toutes les ventes par date pour obtenir une tendance globale.

In [3]:
print("\n🌍 --- Entraînement Modèle Global ---")

# Agrégation
global_train = train_df.groupby('Date')[['Units Sold', 'Price', 'Discount', 'Holiday/Promotion', 'Competitor Pricing']].mean().reset_index()
global_train['Units Sold'] = train_df.groupby('Date')['Units Sold'].sum().values # Somme pour la cible
global_train = global_train.set_index('Date').asfreq('W')

global_test = test_df.groupby('Date')[['Units Sold', 'Price', 'Discount', 'Holiday/Promotion', 'Competitor Pricing']].mean().reset_index()
global_test['Units Sold'] = test_df.groupby('Date')['Units Sold'].sum().values
global_test = global_test.set_index('Date').asfreq('W')

# Entraînement
target_col = 'Units Sold'
exog_cols = ['Price', 'Discount', 'Holiday/Promotion', 'Competitor Pricing']

model_global = SARIMAX(
    global_train[target_col],
    exog=global_train[exog_cols],
    order=(1, 1, 1),
    seasonal_order=(0, 1, 1, 52),
    enforce_stationarity=False,
    enforce_invertibility=False
)
fit_global = model_global.fit(disp=False)

# Évaluation
global_preds = fit_global.forecast(steps=len(global_test), exog=global_test[exog_cols])
rmse_global = np.sqrt(mean_squared_error(global_test[target_col], global_preds))
print(f"✅ Global MAE: {mean_absolute_error(global_test[target_col], global_preds):.2f}")
print(f"✅ Global RMSE: {rmse_global:.2f}")

# Sauvegarde
joblib.dump(fit_global, 'trained_models/global_sarima.pkl')
print("💾 Modèle Global sauvegardé.")


🌍 --- Entraînement Modèle Global ---
✅ Global MAE: 10415.96
✅ Global RMSE: 20944.93
💾 Modèle Global sauvegardé.


## 2. Modèles Par Produit
Nous itérons sur chaque `Product ID`.

In [4]:
print("\n🏭 --- Entraînement Par Produit ---")

products = train_df['Product ID'].unique()
print(f"Produits trouvés: {len(products)}")

for pid in products:
    try:
        # Filtrer
        p_train = train_df[train_df['Product ID'] == pid].set_index('Date').asfreq('W')
        p_test = test_df[test_df['Product ID'] == pid].set_index('Date').asfreq('W')
        
        if len(p_train) < 10:
            continue
            
        # Entraîner (paramètres simplifiés pour rapidité/robustesse)
        # On utilise exog si possible, sinon simple ARIMA si colonnes constantes ou NaN
        exog_train = p_train[exog_cols].fillna(0)
        exog_test = p_test[exog_cols].fillna(0)
        
        model_p = SARIMAX(
            p_train[target_col],
            exog=exog_train,
            order=(1, 1, 1),
            seasonal_order=(0, 0, 0, 0), # Simplification pour éviter erreurs sur petites séries
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        fit_p = model_p.fit(disp=False)
        
        # Save
        joblib.dump(fit_p, f'trained_models/sarima_{pid}.pkl')
        
    except Exception as e:
        print(f"⚠️ Erreur {pid}: {e}")

print("✅ Tous les modèles produits ont été traités.")


🏭 --- Entraînement Par Produit ---
Produits trouvés: 20
✅ Tous les modèles produits ont été traités.
